# 0 Base de datos

In [ ]:
#librerias
from google.cloud import bigquery
import pandas as pd
import time
import logging
import pandas as pd
import json


## 0-1 Datos coingecko

In [ ]:
import requests
import pandas as pd
import time
import numpy as np
from requests.exceptions import RequestException

# -----------------------------------------------
# Configuración
# -----------------------------------------------
COINS_TO_COLLECT = 500
API_COIN_MARKETS_URL = "https://api.coingecko.com/api/v3/coins/markets"
API_MARKET_CHART_URL = "https://api.coingecko.com/api/v3/coins/{}/market_chart"
REQUEST_DELAY = 1.5  # Pausa inicial, se ajusta con reintentos

# -----------------------------------------------
# Funciones de Recolección de Datos
# -----------------------------------------------
def get_crypto_list():
    """Obtiene una lista de las N criptomonedas principales."""
    all_coins = []
    page = 1
    while len(all_coins) < COINS_TO_COLLECT:
        params = {
            "vs_currency": "usd",
            "order": "market_cap_desc",
            "per_page": 250,
            "page": page
        }
        try:
            response = requests.get(API_COIN_MARKETS_URL, params=params)
            response.raise_for_status()
            data = response.json()
            if not data:
                break
            all_coins.extend(data)
            page += 1
            time.sleep(REQUEST_DELAY)
        except RequestException as e:
            print(f"Error al obtener la lista de monedas: {e}")
            break
    return all_coins[:COINS_TO_COLLECT]

def get_historical_features(coin_id, days=7, max_retries=5):
    """
    Obtiene y calcula características históricas para una moneda,
    con reintentos y manejo de errores 429.
    """
    url = API_MARKET_CHART_URL.format(coin_id)
    params = {"vs_currency": "usd", "days": days}

    for retry_count in range(max_retries):
        try:
            response = requests.get(url, params=params)
            if response.status_code == 429:
                # La API nos dice que esperemos.
                wait_time = int(response.headers.get("Retry-After", 60))
                print(f"Demasiadas solicitudes. Esperando {wait_time} segundos...")
                time.sleep(wait_time)
                continue # Reintentar

            response.raise_for_status()
            data = response.json()

            prices = [p[1] for p in data.get("prices", [])]
            volumes = [v[1] for v in data.get("total_volumes", [])]
            market_caps = [m[1] for m in data.get("market_caps", [])]

            if len(prices) < days or len(volumes) < days or len(market_caps) < days:
                return None

            features = {
                'price_initial': prices[0],
                'price_final': prices[-1],
                'price_pct_change_7d': ((prices[-1] - prices[0]) / prices[0]) * 100,
                'trading_volume_7d': np.sum(volumes),
                'market_cap_7d_avg': np.mean(market_caps),
                'vol_to_mcap_ratio': np.sum(volumes) / np.mean(market_caps),
                'price_std_dev_7d': np.std(prices),
                'price_max_7d': np.max(prices),
                'price_min_7d': np.min(prices)
            }
            return features
        except RequestException as e:
            print(f"Error al obtener datos históricos para {coin_id}: {e}")
            if retry_count < max_retries - 1:
                print("Reintentando...")
                time.sleep(2 ** retry_count) # Espera exponencial
            else:
                return None
    return None

# -----------------------------------------------
# Pipeline Principal
# -----------------------------------------------
if __name__ == "__main__":
    print("Iniciando la recolección de datos...")
    coin_list = get_crypto_list()
    if not coin_list:
        print("No se pudo obtener la lista inicial de monedas. Saliendo.")
        exit()

    dataset = []
    for idx, coin in enumerate(coin_list):
        print(f"Procesando moneda {idx+1}/{len(coin_list)}: {coin['id']}")

        market_features = get_historical_features(coin['id'])
        if not market_features:
            continue

        row = {
            'coin_id': coin['id'],
            'symbol': coin['symbol'],
            'name': coin['name'],
            'total_supply': coin.get('total_supply'),
            'circulating_supply': coin.get('circulating_supply'),
            'token_age_days': (pd.Timestamp.now() - pd.to_datetime(coin.get('genesis_date'))).days if coin.get('genesis_date') else None,
            **market_features
        }
        dataset.append(row)
        # Pausa para evitar el límite de tasa incluso con el reintento
        time.sleep(REQUEST_DELAY)

    df = pd.DataFrame(dataset)
    df.to_csv("crypto_market_dataset.csv", index=False)
    print(f"\n Dataset generado con éxito con {len(df)} registros.")

Iniciando la recolección de datos...
Procesando moneda 1/500: bitcoin
Procesando moneda 2/500: ethereum
Procesando moneda 3/500: ripple
Procesando moneda 4/500: tether
Demasiadas solicitudes. Esperando 60 segundos...
Procesando moneda 5/500: solana
Procesando moneda 6/500: binancecoin
Procesando moneda 7/500: usd-coin
Procesando moneda 8/500: dogecoin
Procesando moneda 9/500: staked-ether
Procesando moneda 10/500: cardano
Demasiadas solicitudes. Esperando 60 segundos...
Procesando moneda 11/500: tron
Procesando moneda 12/500: wrapped-steth
Demasiadas solicitudes. Esperando 60 segundos...
Procesando moneda 13/500: chainlink
Procesando moneda 14/500: wrapped-beacon-eth
Procesando moneda 15/500: hyperliquid
Procesando moneda 16/500: wrapped-bitcoin
Demasiadas solicitudes. Esperando 60 segundos...
Procesando moneda 17/500: sui
Procesando moneda 18/500: ethena-usde
Procesando moneda 19/500: avalanche-2
Demasiadas solicitudes. Esperando 60 segundos...
Procesando moneda 20/500: stellar
Proces

In [ ]:
df.describe

<bound method NDFrame.describe of            coin_id symbol           name  total_supply  circulating_supply  \
0          bitcoin    btc        Bitcoin  1.991993e+07        1.991993e+07   
1         ethereum    eth       Ethereum  1.207047e+08        1.207047e+08   
2           ripple    xrp            XRP  9.998581e+10        5.961021e+10   
3           tether   usdt         Tether  1.700017e+11        1.700017e+11   
4           solana    sol         Solana  6.096105e+08        5.424453e+08   
..             ...    ...            ...           ...                 ...   
494    unit-00-rei    rei            Rei  1.000000e+09        1.000000e+09   
495   savings-xdai   sdai   Savings xDAI  9.521845e+07        9.521845e+07   
496  echelon-prime  prime  Echelon Prime  1.111111e+08        5.423475e+07   
497   infinifi-usd   iusd   InfiniFi USD  1.140314e+08        1.140314e+08   
498      avant-usd  avusd      Avant USD  1.141788e+08        1.141788e+08   

    token_age_days  price_ini

## 0-2 Datos Bigquery google

### 0-2-1 feature On-chain

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Conexión con tu credencial JSON
client = bigquery.Client.from_service_account_json("key.json")

# Query mínima funcional: características básicas de 50 tokens distintos
query = """
WITH token_summary AS (
  SELECT
    token_address,
    COUNT(*) AS num_transfers,
    COUNT(DISTINCT from_address) + COUNT(DISTINCT to_address) AS approx_holders,
    MIN(block_timestamp) AS first_seen,
    MAX(block_timestamp) AS last_seen
  FROM `bigquery-public-data.crypto_ethereum.token_transfers`
  GROUP BY token_address
)
SELECT *
FROM token_summary
ORDER BY num_transfers DESC
LIMIT 50
"""

# Ejecutar query y traer resultados a DataFrame
df = client.query(query).to_dataframe()

# Calcular feature adicional: antigüedad en días
df['age_days'] = (pd.to_datetime(df['last_seen']) - pd.to_datetime(df['first_seen'])).dt.days

# Guardar CSV para usar como dataset de prueba
df.to_csv("dataset_tokens_prueba.csv", index=False)

print("CSV generado con éxito: dataset_tokens_prueba.csv")
print(df.head())


CSV generado con éxito: dataset_tokens_prueba.csv
                                token_address  num_transfers  approx_holders  \
0  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2      402779564         6975935   
1  0xdac17f958d2ee523a2206206994597c13d831ec7      351766979        96320277   
2  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48      171087577        39228183   
3  0x06450dee7fd2fb8e39061434babcfc05599a6fb8       44376023        12383080   
4  0x6b175474e89094c44da98b954eedeac495271d0f       24569101         4966372   

                 first_seen                 last_seen  age_days  
0 2017-12-18 09:07:58+00:00 2025-09-14 08:33:23+00:00      2826  
1 2017-11-28 15:38:10+00:00 2025-09-14 08:33:23+00:00      2846  
2 2018-09-10 18:26:41+00:00 2025-09-14 08:33:23+00:00      2560  
3 2022-10-08 18:03:59+00:00 2025-09-14 08:33:11+00:00      1071  
4 2019-11-13 21:22:33+00:00 2025-09-14 08:33:23+00:00      2131  


In [ ]:
from google.cloud import bigquery

client = bigquery.Client.from_service_account_json("key.json")

# Query para ver las columnas reales de la tabla tokens
query = """
SELECT *
FROM `bigquery-public-data.crypto_ethereum.tokens`
LIMIT 5
"""

df = client.query(query).to_dataframe()

print("✅ Preview de tokens:")
print(df.head())
print("\n📌 Columnas disponibles:")
print(df.columns.tolist())


✅ Preview de tokens:
                                      address symbol       name decimals  \
0  0x81a7d1f0c28189a0b083521d5e3ac4b6c04149ad   DOPE  Dope Boyz     None   
1  0x461c5448a753b70fe509c67d8c5954ac1112105c   GLDC   GoldCoin       18   
2  0x3b987b27992fe7f665b3f43d8485cafbec8fcbba  sosal     sosal?       18   
3  0x2724b281344acb7536439aabd018ac8bd980d2fb    YZY        YZY        9   
4  0xdc34889e6255be40373e2b9410782240a466b7e8    YZY        YZY        9   

                   total_supply           block_timestamp  block_number  \
0                             1 2025-08-24 16:19:59+00:00      23212101   
1  1000000000000000000000000000 2025-08-24 23:30:23+00:00      23214248   
2                             0 2025-08-20 19:26:35+00:00      23184372   
3            100000000000000000 2025-08-21 04:38:11+00:00      23187120   
4            100000000000000000 2025-08-21 04:30:35+00:00      23187082   

                                          block_hash  
0  0xfb6b66a487d

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Cliente
client = bigquery.Client.from_service_account_json("key.json")

# Query mínima para prueba de tipos y fechas
query_test = """
SELECT
  token_address,
  MIN(SAFE_CAST(block_timestamp AS TIMESTAMP)) AS first_seen,
  MAX(SAFE_CAST(block_timestamp AS TIMESTAMP)) AS last_seen,
  COUNT(*) AS num_transfers
FROM `bigquery-public-data.crypto_ethereum.token_transfers`
GROUP BY token_address
LIMIT 5
"""

df_test = client.query(query_test).to_dataframe()

# Mostrar tipos y data
print("✅ Tipos de columnas:")
print(df_test.dtypes)
print("\n✅ Preview de datos:")
print(df_test)


✅ Tipos de columnas:
token_address                 object
first_seen       datetime64[us, UTC]
last_seen        datetime64[us, UTC]
num_transfers                  Int64
dtype: object

✅ Preview de datos:
                                token_address                first_seen  \
0  0xd0d6d6c5fe4a677d343cc433536bb717bae167dd 2017-06-19 19:53:58+00:00   
1  0x3fbabe48e137c9ef4f8cb517a4762add7e6242ae 2024-06-29 04:28:59+00:00   
2  0xa1afffe3f4d611d252010e3eaf6f4d77088b0cd7 2020-11-18 21:25:26+00:00   
3  0xfa99a87b14b02e2240c79240c5a20f945ca5ef76 2020-12-07 19:04:54+00:00   
4  0xd06b49243b3b0a873b51b8636cc138832532d7fa 2020-11-29 01:24:51+00:00   

                  last_seen  num_transfers  
0 2025-09-02 13:18:11+00:00          17818  
1 2025-09-14 21:12:35+00:00          14347  
2 2025-09-09 07:56:47+00:00          75384  
3 2025-09-14 14:02:35+00:00          20584  
4 2024-04-11 10:31:59+00:00           1653  


In [ ]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client.from_service_account_json("key.json")

# Query corregida con agregación adecuada
query = """
WITH token_summary AS (
  SELECT
    token_address,
    COUNT(*) AS num_transfers,
    COUNT(DISTINCT from_address) + COUNT(DISTINCT to_address) AS approx_holders,
    MIN(block_timestamp) AS first_seen,
    MAX(block_timestamp) AS last_seen
  FROM `bigquery-public-data.crypto_ethereum.token_transfers`
  GROUP BY token_address
),
top10_holders AS (
  SELECT
    token_address,
    SUM(total_value) AS top10_balance
  FROM (
    SELECT
      token_address,
      to_address,
      SUM(SAFE_CAST(value AS FLOAT64)) AS total_value,
      ROW_NUMBER() OVER(
        PARTITION BY token_address
        ORDER BY SUM(SAFE_CAST(value AS FLOAT64)) DESC
      ) AS rn
    FROM `bigquery-public-data.crypto_ethereum.token_transfers`
    GROUP BY token_address, to_address
  )
  WHERE rn <= 10
  GROUP BY token_address
),
daily_stats AS (
  SELECT
    token_address,
    AVG(tx_count) AS avg_transfers_per_day,
    STDDEV(tx_count) AS std_transfers_per_day,
    COUNT(*) AS active_days
  FROM (
    SELECT
      token_address,
      DATE(block_timestamp) AS day,
      COUNT(*) AS tx_count
    FROM `bigquery-public-data.crypto_ethereum.token_transfers`
    GROUP BY token_address, day
  )
  GROUP BY token_address
)
SELECT
  ts.token_address AS address,
  t.symbol,
  t.name,
  ts.num_transfers,
  ts.approx_holders,
  ts.first_seen,
  ts.last_seen,
  SAFE_DIVIDE(th.top10_balance, ts.num_transfers) AS top10_concentration_proxy,
  ds.avg_transfers_per_day,
  ds.std_transfers_per_day,
  ds.active_days
FROM token_summary ts
LEFT JOIN top10_holders th ON ts.token_address = th.token_address
LEFT JOIN daily_stats ds ON ts.token_address = ds.token_address
LEFT JOIN `bigquery-public-data.crypto_ethereum.tokens` t
  ON ts.token_address = t.address
ORDER BY ts.num_transfers DESC
LIMIT 10
"""

# Ejecutar query
df = client.query(query).to_dataframe()

# Convertir timestamps a datetime en Python
df['first_seen'] = pd.to_datetime(df['first_seen'])
df['last_seen'] = pd.to_datetime(df['last_seen'])

# Calcular métricas derivadas en Python
df['age_days'] = (df['last_seen'] - df['first_seen']).dt.days
df['inactive_days_proxy'] = df['age_days'] - df['active_days']

# Guardar CSV
df.to_csv("dataset_tokens_extendido.csv", index=False)

print("✅ CSV generado: dataset_tokens_extendido.csv")
print(df.head(10))

✅ CSV generado: dataset_tokens_extendido.csv
                                      address symbol             name  \
0  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2   WETH    Wrapped Ether   
1  0xdac17f958d2ee523a2206206994597c13d831ec7   USDT       Tether USD   
2  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48   None             None   
3  0x06450dee7fd2fb8e39061434babcfc05599a6fb8   None             None   
4  0x6b175474e89094c44da98b954eedeac495271d0f   None             None   
5  0x514910771af9ca656af840dff83e8264ecf986ca   LINK  ChainLink Token   
6  0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce   SHIB        SHIBA INU   
7  0x2260fac5e5542a773aa44fbcfedf7c193bc2c599   WBTC      Wrapped BTC   
8  0x0000000000a39bb272e79075ade125fd351887ac   None             None   
9  0x174bfa6600bf90c885c7c01c7031389ed1461ab9    MGC   More Gold Coin   

   num_transfers  approx_holders                first_seen  \
0      402961481         6977804 2017-12-18 09:07:58+00:00   
1      351975575        963

In [ ]:
import pandas as pd

# Cargar dataset generado
df = pd.read_csv("dataset_tokens_extendido.csv")

# Convertir timestamps a datetime
df['first_seen'] = pd.to_datetime(df['first_seen'])
df['last_seen'] = pd.to_datetime(df['last_seen'])

# --- Función de análisis de features ---
def feature_summary(df):
    summary = []

    for col in df.columns:
        dtype = df[col].dtype
        n_unique = df[col].nunique()
        n_missing = df[col].isna().sum()
        perc_missing = 100 * n_missing / len(df)
        min_val = df[col].min() if pd.api.types.is_numeric_dtype(df[col]) else None
        max_val = df[col].max() if pd.api.types.is_numeric_dtype(df[col]) else None
        mean_val = df[col].mean() if pd.api.types.is_numeric_dtype(df[col]) else None
        summary.append({
            'feature': col,
            'dtype': dtype,
            'n_unique': n_unique,
            'n_missing': n_missing,
            'perc_missing': perc_missing,
            'min': min_val,
            'max': max_val,
            'mean': mean_val
        })

    return pd.DataFrame(summary)

# Generar resumen
feature_info = feature_summary(df)

# Mostrar resumen
print("✅ Resumen de features:")
print(feature_info)

# --- Opcional: revisar correlación entre features numéricas ---
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
corr_matrix = df[num_cols].corr()

print("\n✅ Matriz de correlación:")
print(corr_matrix)

# --- Opcional: guardar resumen a CSV ---
feature_info.to_csv("feature_summary.csv", index=False)
corr_matrix.to_csv("feature_correlation.csv")


✅ Resumen de features:
                      feature                dtype  n_unique  n_missing  \
0                     address               object        10          0   
1                      symbol               object         6          4   
2                        name               object         6          4   
3               num_transfers                int64        10          0   
4              approx_holders                int64        10          0   
5                  first_seen  datetime64[ns, UTC]        10          0   
6                   last_seen  datetime64[ns, UTC]         6          0   
7   top10_concentration_proxy              float64        10          0   
8       avg_transfers_per_day              float64        10          0   
9       std_transfers_per_day              float64        10          0   
10                active_days                int64        10          0   
11                   age_days                int64        10          0   
12

## 0-3 Consultas finales

### 0-3-1 consulta final coingecko "id", "symbol", "name", "current_price", "market_cap", "total_volume"

In [ ]:
import requests
import pandas as pd
import time

vs_currency = "usd"
per_page = 250
max_pages = 60  # Limite máximo de páginas para Coingecko
all_coins = []

for page in range(1, max_pages + 1):
    print(f"✅ Descargando página {page}/{max_pages}...")

    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": vs_currency,
        "order": "market_cap_desc",
        "per_page": per_page,
        "page": page,
        "sparkline": "false"
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
    except Exception as e:
        print(f"❌ Error en request o parseo JSON en página {page}: {e}")
        break

    # Validar que venga una lista
    if isinstance(data, list) and data:
        all_coins.extend(data)
    else:
        print(f"❌ Respuesta inesperada o vacía en página {page}: {data}")
        break

    # Rate limit seguro: Coingecko permite ~30 llamadas/min, aquí 2 seg entre llamadas
    time.sleep(20)

if all_coins:
    df = pd.DataFrame(all_coins)
    # Seleccionar solo columnas relevantes
    df = df[["id", "symbol", "name", "current_price", "market_cap", "total_volume"]]
    df.to_csv("coingecko_tokens2.csv", index=False)
    print(f"✅ Guardado CSV con {len(df)} tokens: coingecko_tokens2.csv")
else:
    print("❌ No se descargaron tokens")


✅ Descargando página 1/60...
✅ Descargando página 2/60...
✅ Descargando página 3/60...
✅ Descargando página 4/60...
✅ Descargando página 5/60...
✅ Descargando página 6/60...
✅ Descargando página 7/60...
✅ Descargando página 8/60...
✅ Descargando página 9/60...
✅ Descargando página 10/60...
✅ Descargando página 11/60...
✅ Descargando página 12/60...
✅ Descargando página 13/60...
✅ Descargando página 14/60...
✅ Descargando página 15/60...
✅ Descargando página 16/60...
✅ Descargando página 17/60...
✅ Descargando página 18/60...
✅ Descargando página 19/60...
✅ Descargando página 20/60...
✅ Descargando página 21/60...
✅ Descargando página 22/60...
✅ Descargando página 23/60...
✅ Descargando página 24/60...
✅ Descargando página 25/60...
✅ Descargando página 26/60...
✅ Descargando página 27/60...
✅ Descargando página 28/60...
✅ Descargando página 29/60...
✅ Descargando página 30/60...
✅ Descargando página 31/60...
✅ Descargando página 32/60...
✅ Descargando página 33/60...
✅ Descargando págin

In [ ]:
print(df)

                     id symbol              name  current_price    market_cap  \
0               bitcoin    btc           Bitcoin  115432.000000  2.301122e+12   
1              ethereum    eth          Ethereum    4611.930000  5.566856e+11   
2                ripple    xrp               XRP       3.030000  1.808420e+11   
3                tether   usdt            Tether       1.001000  1.703268e+11   
4                solana    sol            Solana     241.230000  1.309971e+11   
...                 ...    ...               ...            ...           ...   
14995           num-ars   nars           Num ARS       0.000676  0.000000e+00   
14996    purple-bitcoin   pbtc    Purple Bitcoin       0.359607  0.000000e+00   
14997         ponke-ton  ponke         Ponke TON       0.000077  0.000000e+00   
14998              pola   pola      Pola On Base       0.000050  0.000000e+00   
14999  mxmboxceus-token    mbe  MxmBoxcEus Token       0.026741  0.000000e+00   

       total_volume  
0    

### 0-3-2 consulta final bigquery optimizado

In [ ]:
from google.cloud import bigquery
import pandas as pd
import time
import logging

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Cliente BigQuery
client = bigquery.Client.from_service_account_json("key.json")

# Parámetros
BATCH_SIZE = 100
MAX_BYTES_BILLED = 500 * 1024**3  # 500 GB
MIN_TRANSFERS = 10  # Filtrar tokens con menos de 10 transfers
MAX_TOKENS = 500    # Total tokens para tu análisis

# Paso 1: Obtener tokens válidos con filtro desde el inicio
logger.info("Obteniendo tokens válidos filtrados por transferencias...")
tokens_query = f"""
WITH token_counts AS (
  SELECT
    t.address,
    COUNT(tt.token_address) AS num_transfers
  FROM `bigquery-public-data.crypto_ethereum.tokens` t
  LEFT JOIN `bigquery-public-data.crypto_ethereum.token_transfers` tt
    ON t.address = tt.token_address
  WHERE t.symbol IS NOT NULL AND t.name IS NOT NULL
  GROUP BY t.address
  HAVING COUNT(tt.token_address) >= {MIN_TRANSFERS}
)
SELECT address
FROM token_counts
LIMIT {MAX_TOKENS}
"""
tokens_df = client.query(tokens_query).to_dataframe()
tokens = tokens_df['address'].tolist()
logger.info(f"Encontrados {len(tokens)} tokens válidos y útiles")

# Paso 2: Procesar tokens en batches
all_results = []
for i in range(0, len(tokens), BATCH_SIZE):
    batch = tokens[i:i + BATCH_SIZE]
    token_list = ", ".join([f"'{t}'" for t in batch])

    query = f"""
    WITH token_summary AS (
      SELECT
        token_address,
        COUNT(*) AS num_transfers,
        COUNT(DISTINCT from_address) + COUNT(DISTINCT to_address) AS approx_holders,
        MIN(block_timestamp) AS first_seen,
        MAX(block_timestamp) AS last_seen
      FROM `bigquery-public-data.crypto_ethereum.token_transfers`
      WHERE token_address IN ({token_list})
      GROUP BY token_address
    ),
    top10_holders AS (
      SELECT
        token_address,
        SUM(total_value) AS top10_balance
      FROM (
        SELECT
          token_address,
          to_address,
          SUM(SAFE_CAST(value AS FLOAT64)) AS total_value,
          ROW_NUMBER() OVER(
            PARTITION BY token_address
            ORDER BY SUM(SAFE_CAST(value AS FLOAT64)) DESC
          ) AS rn
        FROM `bigquery-public-data.crypto_ethereum.token_transfers`
        WHERE token_address IN ({token_list})
        GROUP BY token_address, to_address
      )
      WHERE rn <= 10
      GROUP BY token_address
    ),
    daily_stats AS (
      SELECT
        token_address,
        AVG(tx_count) AS avg_transfers_per_day,
        STDDEV(tx_count) AS std_transfers_per_day,
        COUNT(*) AS active_days
      FROM (
        SELECT
          token_address,
          DATE(block_timestamp) AS day,
          COUNT(*) AS tx_count
        FROM `bigquery-public-data.crypto_ethereum.token_transfers`
        WHERE token_address IN ({token_list})
        GROUP BY token_address, day
      )
      GROUP BY token_address
    )
    SELECT
      ts.token_address AS address,
      t.symbol,
      t.name,
      ts.num_transfers,
      ts.approx_holders,
      ts.first_seen,
      ts.last_seen,
      SAFE_DIVIDE(th.top10_balance, ts.num_transfers) AS top10_concentration_proxy,
      ds.avg_transfers_per_day,
      ds.std_transfers_per_day,
      ds.active_days
    FROM token_summary ts
    LEFT JOIN top10_holders th ON ts.token_address = th.token_address
    LEFT JOIN daily_stats ds ON ts.token_address = ds.token_address
    LEFT JOIN `bigquery-public-data.crypto_ethereum.tokens` t
      ON ts.token_address = t.address
    """

    try:
        df_batch = client.query(query, job_config=bigquery.QueryJobConfig(
            maximum_bytes_billed=MAX_BYTES_BILLED
        )).to_dataframe()

        if not df_batch.empty:
            all_results.append(df_batch)
            logger.info(f"Lote {i//BATCH_SIZE + 1} procesado, {len(df_batch)} tokens")
        time.sleep(1)  # Pausa para proteger recursos
    except Exception as e:
        logger.error(f"Error en lote {i//BATCH_SIZE + 1}: {e}")

# Paso 3: Combinar resultados finales
if all_results:
    df = pd.concat(all_results, ignore_index=True)
    df['first_seen'] = pd.to_datetime(df['first_seen'])
    df['last_seen'] = pd.to_datetime(df['last_seen'])
    df['age_days'] = (df['last_seen'] - df['first_seen']).dt.days
    df['inactive_days_proxy'] = df['age_days'] - df['active_days']

    df.to_csv("token_analysis_filtered.csv", index=False)
    logger.info(f"✅ CSV generado: token_analysis_filtered.csv con {len(df)} tokens")
else:
    logger.warning("❌ No se obtuvieron resultados")


INFO:__main__:Obteniendo tokens válidos filtrados por transferencias...
INFO:__main__:Encontrados 500 tokens válidos y útiles
INFO:__main__:Lote 1 procesado, 100 tokens
INFO:__main__:Lote 2 procesado, 100 tokens
INFO:__main__:Lote 3 procesado, 100 tokens
INFO:__main__:Lote 4 procesado, 100 tokens
INFO:__main__:Lote 5 procesado, 100 tokens
INFO:__main__:✅ CSV generado: token_analysis_filtered.csv con 500 tokens


### 0-3-2-1 Uso de GCP
En GCP se gasto 5 query de Tiempo total de ranura
2 h 37 min
Duración
Bytes procesados
403.92 GB
Registros leídos: 2700610582
Registros escritos: 17202
cada query o id de trabajo.

analogo al batch de 200 por eso para la query final se sube el tamaño de batch dee 100 a 250.

In [ ]:


# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Cliente BigQuery
client = bigquery.Client.from_service_account_json("key.json")

# Parámetros
BATCH_SIZE = 250
MAX_BYTES_BILLED = 500 * 1024**3  # 500 GB
MIN_TRANSFERS = 10  # Filtrar tokens con menos de 10 transfers
MAX_TOKENS = 5000    # Total tokens para tu análisis

# Paso 1: Obtener tokens válidos con filtro desde el inicio
logger.info("Obteniendo tokens válidos filtrados por transferencias...")
tokens_query = f"""
WITH token_counts AS (
  SELECT
    t.address,
    COUNT(tt.token_address) AS num_transfers
  FROM `bigquery-public-data.crypto_ethereum.tokens` t
  LEFT JOIN `bigquery-public-data.crypto_ethereum.token_transfers` tt
    ON t.address = tt.token_address
  WHERE t.symbol IS NOT NULL AND t.name IS NOT NULL
  GROUP BY t.address
  HAVING COUNT(tt.token_address) >= {MIN_TRANSFERS}
)
SELECT address
FROM token_counts
LIMIT {MAX_TOKENS}
"""
tokens_df = client.query(tokens_query).to_dataframe()
tokens = tokens_df['address'].tolist()
logger.info(f"Encontrados {len(tokens)} tokens válidos y útiles")

# Paso 2: Procesar tokens en batches
all_results = []
for i in range(0, len(tokens), BATCH_SIZE):
    batch = tokens[i:i + BATCH_SIZE]
    token_list = ", ".join([f"'{t}'" for t in batch])

    query = f"""
    WITH token_summary AS (
      SELECT
        token_address,
        COUNT(*) AS num_transfers,
        COUNT(DISTINCT from_address) + COUNT(DISTINCT to_address) AS approx_holders,
        MIN(block_timestamp) AS first_seen,
        MAX(block_timestamp) AS last_seen
      FROM `bigquery-public-data.crypto_ethereum.token_transfers`
      WHERE token_address IN ({token_list})
      GROUP BY token_address
    ),
    top10_holders AS (
      SELECT
        token_address,
        SUM(total_value) AS top10_balance
      FROM (
        SELECT
          token_address,
          to_address,
          SUM(SAFE_CAST(value AS FLOAT64)) AS total_value,
          ROW_NUMBER() OVER(
            PARTITION BY token_address
            ORDER BY SUM(SAFE_CAST(value AS FLOAT64)) DESC
          ) AS rn
        FROM `bigquery-public-data.crypto_ethereum.token_transfers`
        WHERE token_address IN ({token_list})
        GROUP BY token_address, to_address
      )
      WHERE rn <= 10
      GROUP BY token_address
    ),
    daily_stats AS (
      SELECT
        token_address,
        AVG(tx_count) AS avg_transfers_per_day,
        STDDEV(tx_count) AS std_transfers_per_day,
        COUNT(*) AS active_days
      FROM (
        SELECT
          token_address,
          DATE(block_timestamp) AS day,
          COUNT(*) AS tx_count
        FROM `bigquery-public-data.crypto_ethereum.token_transfers`
        WHERE token_address IN ({token_list})
        GROUP BY token_address, day
      )
      GROUP BY token_address
    )
    SELECT
      ts.token_address AS address,
      t.symbol,
      t.name,
      ts.num_transfers,
      ts.approx_holders,
      ts.first_seen,
      ts.last_seen,
      SAFE_DIVIDE(th.top10_balance, ts.num_transfers) AS top10_concentration_proxy,
      ds.avg_transfers_per_day,
      ds.std_transfers_per_day,
      ds.active_days
    FROM token_summary ts
    LEFT JOIN top10_holders th ON ts.token_address = th.token_address
    LEFT JOIN daily_stats ds ON ts.token_address = ds.token_address
    LEFT JOIN `bigquery-public-data.crypto_ethereum.tokens` t
      ON ts.token_address = t.address
    """

    try:
        df_batch = client.query(query, job_config=bigquery.QueryJobConfig(
            maximum_bytes_billed=MAX_BYTES_BILLED
        )).to_dataframe()

        if not df_batch.empty:
            all_results.append(df_batch)
            logger.info(f"Lote {i//BATCH_SIZE + 1} procesado, {len(df_batch)} tokens")
        time.sleep(1)  # Pausa para proteger recursos
    except Exception as e:
        logger.error(f"Error en lote {i//BATCH_SIZE + 1}: {e}")

# Paso 3: Combinar resultados finales
if all_results:
    df = pd.concat(all_results, ignore_index=True)
    df['first_seen'] = pd.to_datetime(df['first_seen'])
    df['last_seen'] = pd.to_datetime(df['last_seen'])
    df['age_days'] = (df['last_seen'] - df['first_seen']).dt.days
    df['inactive_days_proxy'] = df['age_days'] - df['active_days']

    df.to_csv("bd_onchain_final.csv", index=False)
    logger.info(f"✅ CSV generado: bd_onchain_final.csv con {len(df)} tokens")
else:
    logger.warning("❌ No se obtuvieron resultados")


INFO:__main__:Obteniendo tokens válidos filtrados por transferencias...
INFO:__main__:Encontrados 5000 tokens válidos y útiles
INFO:__main__:Lote 1 procesado, 250 tokens
INFO:__main__:Lote 2 procesado, 250 tokens
INFO:__main__:Lote 3 procesado, 250 tokens
INFO:__main__:Lote 4 procesado, 250 tokens
INFO:__main__:Lote 5 procesado, 250 tokens
INFO:__main__:Lote 6 procesado, 250 tokens
INFO:__main__:Lote 7 procesado, 250 tokens
INFO:__main__:Lote 8 procesado, 250 tokens
INFO:__main__:Lote 9 procesado, 250 tokens
INFO:__main__:Lote 10 procesado, 250 tokens
INFO:__main__:Lote 11 procesado, 250 tokens
INFO:__main__:Lote 12 procesado, 250 tokens
INFO:__main__:Lote 13 procesado, 250 tokens
INFO:__main__:Lote 14 procesado, 250 tokens
INFO:__main__:Lote 15 procesado, 250 tokens
INFO:__main__:Lote 16 procesado, 250 tokens
INFO:__main__:Lote 17 procesado, 250 tokens
INFO:__main__:Lote 18 procesado, 250 tokens
INFO:__main__:Lote 19 procesado, 250 tokens
INFO:__main__:Lote 20 procesado, 250 tokens
IN

In [ ]:
import pandas as pd

# Cargar dataset generado
df = pd.read_csv("bd_onchain_final.csv")

# Convertir timestamps a datetime
df['first_seen'] = pd.to_datetime(df['first_seen'])
df['last_seen'] = pd.to_datetime(df['last_seen'])

# --- Función de análisis de features ---
def feature_summary(df):
    summary = []

    for col in df.columns:
        dtype = df[col].dtype
        n_unique = df[col].nunique()
        n_missing = df[col].isna().sum()
        perc_missing = 100 * n_missing / len(df)
        min_val = df[col].min() if pd.api.types.is_numeric_dtype(df[col]) else None
        max_val = df[col].max() if pd.api.types.is_numeric_dtype(df[col]) else None
        mean_val = df[col].mean() if pd.api.types.is_numeric_dtype(df[col]) else None
        summary.append({
            'feature': col,
            'dtype': dtype,
            'n_unique': n_unique,
            'n_missing': n_missing,
            'perc_missing': perc_missing,
            'min': min_val,
            'max': max_val,
            'mean': mean_val
        })

    return pd.DataFrame(summary)

# Generar resumen
feature_info = feature_summary(df)

# Mostrar resumen
print("✅ Resumen de features:")
print(feature_info)

# --- Opcional: revisar correlación entre features numéricas ---
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
corr_matrix = df[num_cols].corr()

print("\n✅ Matriz de correlación:")
print(corr_matrix)




✅ Resumen de features:
                      feature                dtype  n_unique  n_missing  \
0                     address               object      5000          0   
1                      symbol               object      4323          0   
2                        name               object      4829          0   
3               num_transfers                int64      1655          0   
4              approx_holders                int64      1414          0   
5                  first_seen  datetime64[ns, UTC]      4993          0   
6                   last_seen  datetime64[ns, UTC]      4945          0   
7   top10_concentration_proxy              float64      4968          0   
8       avg_transfers_per_day              float64      3131          0   
9       std_transfers_per_day              float64      3875        446   
10                active_days                int64       569          0   
11                   age_days                int64      1683          0   
12

In [ ]:
df.head(10)

,address,symbol,name,num_transfers,approx_holders,first_seen,last_seen,top10_concentration_proxy,avg_transfers_per_day,std_transfers_per_day,active_days,age_days,inactive_days_proxy
0,0x2a2c06306f1352b7e742bb09eb421f91ffa2791a,TBL,TBL,114,86,2018-10-30 08:39:13+00:00,2021-01-13 23:18:44+00:00,9.112790e+24,2.850000,2.402456,40,806,766
1,0x9b613bb970ef1f8dd15f9da863fe6b059c20dfb6,CYC,CYC,11,10,2018-04-20 10:03:19+00:00,2018-05-10 05:20:41+00:00,1.482727e+24,1.571429,1.133893,7,19,12
2,0x74004a7227615fb52b82d17ffabfa376907d8a4d,AVM,AVM Tokens,54,46,2018-04-26 04:15:44+00:00,2021-06-30 03:01:56+00:00,1.386268e+24,1.350000,0.769615,40,1160,1120
3,0x05271b76b10c785358f531281138e2b296821905,CHZ,Cholyz,11,5,2019-08-10 00:03:56+00:00,2019-08-10 04:23:01+00:00,1.480000e+10,11.000000,NaN,1,0,-1
4,0x3c4bea627039f0b7e7d21e34bb9c9fe962977518,UCT,UCOT,26616,22887,2018-01-30 04:45:34+00:00,2025-09-11 01:45:59+00:00,1.686164e+23,25.666345,226.132817,1037,2780,1743
5,0xd964d2ddba0c50e535d09b80630cb8cd88c5c9ef,CT-💯,💯,18,10,2018-02-21 20:32:30+00:00,2018-03-03 00:48:54+00:00,1.000000e+08,6.000000,6.928203,3,9,6
6,0x3a358e34c7774a8cd7c916a594454701f27f380e,AT,AutoToken,2710,2748,2018-07-05 17:16:20+00:00,2023-03-25 02:33:59+00:00,3.787799e+19,31.511628,198.834175,86,1723,1637
7,0x1d5bd914b57aeb1ce91d37d8454dfa48be22eb60,RGX15,RGX (x15) Token,12,15,2017-09-12 17:15:43+00:00,2018-06-28 11:06:23+00:00,7.916583e+03,1.333333,0.707107,9,288,279
8,0x9750ce7823568c9354720407e2763fd16bf0b1c6,FMC,FOMO,30925,22174,2019-05-10 01:53:48+00:00,2023-10-26 15:03:47+00:00,1.791051e+21,222.482014,574.330516,139,1630,1491
9,0xa18e33b88312a14e5f34253152540e36bac279b3,UVCN,UVCN,28,30,2018-03-22 11:13:08+00:00,2018-09-30 10:59:41+00:00,1.428572e+27,1.750000,0.930949,16,191,175


## 0-4 BD fraudes

In [ ]:

# 1Cargar diccionario de tokens de TokenScout
with open("token-dict.json", "r", encoding="utf-8") as f:
    token_dict = json.load(f)

# Convertir a DataFrame: address | token_id
df_tokens = pd.DataFrame(list(token_dict.items()), columns=["address", "token_id"])

# Cargar CSV de etiquetas procesadas: tokens | t1-labels | t2-labels
df_labels = pd.read_csv("token_labels_processed.csv")  # tokens = id

# Hacer merge de etiquetas con diccionario de tokens por token_id
df_labels = df_labels.merge(df_tokens, left_on="tokens", right_on="token_id", how="left")

# Crear columna organization y eliminar columnas intermedias
df_labels["organization"] = "TokenScout"
df_final = df_labels[["address", "organization", "t1-labels", "t2-labels"]]

# Guardar CSV limpio
df_final.to_csv("tokenscout_labels_clean.csv", index=False)

print("✅ CSV TokenScout limpio creado: tokenscout_labels_clean.csv")


✅ CSV TokenScout limpio creado: tokenscout_labels_clean.csv


In [ ]:
import pandas as pd

# Cargar archivo principal
df_main = pd.read_csv("bd_onchain_final.csv", dtype=str)

# 2Cargar CSV limpio de TokenScout
df_tokenscout = pd.read_csv("tokenscout_labels_clean.csv", dtype=str)

# 3Merge izquierdo por address
df_merged = df_main.merge(
    df_tokenscout,
    on="address",
    how="left"
)

# Rellenar valores faltantes con default
df_merged["organization"] = df_merged["organization"].fillna("0")
df_merged["t1-labels"] = df_merged["t1-labels"].fillna("0")
df_merged["t2-labels"] = df_merged["t2-labels"].fillna("0")
new_cols = ["organization", "t1-labels", "t2-labels"]

for col in new_cols:
    print(f"--- {col} ---")
    print(f"Cantidad de valores únicos: {df_merged[col].nunique()}")
    print(f"Cantidad de registros con 0: {(df_merged[col] == '0').sum()}")
    print(f"Distribución de valores:\n{df_merged[col].value_counts()}\n")

# Guardar CSV final
df_merged.to_csv("bd_onchain_final_with_tokenscout.csv", index=False)

print("✅ Archivo principal combinado con TokenScout creado.")



--- organization ---
Cantidad de valores únicos: 2
Cantidad de registros con 0: 288
Distribución de valores:
organization
TokenScout    4712
0              288
Name: count, dtype: int64

--- t1-labels ---
Cantidad de valores únicos: 3
Cantidad de registros con 0: 288
Distribución de valores:
t1-labels
scam           4542
0               288
truthworthy     170
Name: count, dtype: int64

--- t2-labels ---
Cantidad de valores únicos: 5
Cantidad de registros con 0: 288
Distribución de valores:
t2-labels
rugpull        3352
honeypot        732
ponzi           458
0               288
truthworthy     170
Name: count, dtype: int64

✅ Archivo principal combinado con TokenScout creado.


In [ ]:

# Cargar archivos
df_main = pd.read_csv("bd_onchain_final_with_tokenscout.csv", dtype=str)
df_coingecko = pd.read_csv("coingecko_tokens2.csv", dtype=str)

# Limpiar nombres de columnas por si hay espacios
df_main.columns = df_main.columns.str.strip()
df_coingecko.columns = df_coingecko.columns.str.strip()

# Columnas de Coingecko a agregar
coingecko_cols = [col for col in df_coingecko.columns if col != "symbol"]

# Merge izquierdo por 'symbol'
df_final = df_main.merge(
    df_coingecko[["symbol"] + coingecko_cols],
    on="symbol",
    how="left"
)

# Verifica qué columnas realmente están en df_final
print("Columnas en df_final:", df_final.columns.tolist())

# Rellenar valores faltantes solo para columnas que existen
for col in coingecko_cols:
    if col in df_final.columns:
        df_final[col] = df_final[col].fillna("0")
    else:
        print(f"⚠️ Columna {col} no existe en df_final, se omite.")

# Guardar CSV final
df_final.to_csv("bd_onchain_final_with_tokenscout_coingecko.csv", index=False)
print("✅ Archivo final combinado creado correctamente.")


Columnas en df_final: ['address', 'symbol', 'name_x', 'num_transfers', 'approx_holders', 'first_seen', 'last_seen', 'top10_concentration_proxy', 'avg_transfers_per_day', 'std_transfers_per_day', 'active_days', 'age_days', 'inactive_days_proxy', 'organization', 't1-labels', 't2-labels', 'id', 'name_y', 'current_price', 'market_cap', 'total_volume']
⚠️ Columna name no existe en df_final, se omite.
✅ Archivo final combinado creado correctamente.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Crear carpeta para gráficos
os.makedirs("graficos_bd_onchain", exist_ok=True)

# Cargar CSV final
df = pd.read_csv("bd_onchain_final_with_tokenscout_coingecko.csv", dtype=str)
df2=pd.read_csv("coingecko_tokens2.csv")
summary = []

# Columnas categóricas y numéricas a graficar
cat_cols = ['organization', 't1-labels', 't2-labels']
num_cols = ['num_transfers', 'approx_holders', 'top10_concentration_proxy',
            'avg_transfers_per_day', 'std_transfers_per_day', 'active_days',
            'age_days', 'inactive_days_proxy', 'current_price', 'market_cap', 'total_volume']

# Analizar todas las columnas
for col in df.columns:
    col_data = df[col]

    # Intentar convertir a numérico
    col_numeric = pd.to_numeric(col_data, errors='coerce')

    summary.append({
        "column": col,
        "dtype": col_data.dtype,
        "n_unique": col_data.nunique(),
        "n_null_or_empty": col_data.isna().sum() + (col_data == '').sum(),
        "top_5_values": col_data.value_counts().head(5).to_dict(),
        "mean": col_numeric.replace([np.inf, -np.inf], np.nan).mean(),
        "std": col_numeric.replace([np.inf, -np.inf], np.nan).std(),
        "min": col_numeric.replace([np.inf, -np.inf], np.nan).min(),
        "max": col_numeric.replace([np.inf, -np.inf], np.nan).max(),
        "median": col_numeric.replace([np.inf, -np.inf], np.nan).median()
    })

    # Graficar categóricas relevantes
    if col in cat_cols:
        counts = col_data.value_counts()
        plt.figure(figsize=(8,4))
        counts.plot(kind='bar', color='skyblue')
        plt.title(f"Distribución de {col}")
        plt.ylabel("Frecuencia")
        plt.tight_layout()
        plt.savefig(f"graficos_bd_onchain/{col}_bar.png")
        plt.close()

    # Graficar numéricas relevantes
    elif col in num_cols:
        numeric_clean = col_numeric.replace([np.inf, -np.inf], np.nan).dropna()
        if not numeric_clean.empty:
            plt.figure(figsize=(8,4))
            plt.hist(numeric_clean, bins=50, color='lightgreen', edgecolor='black')
            plt.title(f"Histograma de {col}")
            plt.xlabel(col)
            plt.ylabel("Frecuencia")
            plt.tight_layout()
            plt.savefig(f"graficos_bd_onchain/{col}_hist.png")
            plt.close()

# Guardar resumen en CSV
df_summary = pd.DataFrame(summary)
df_summary.to_csv("resumen_estadistico_completo_bd_onchain.csv", index=False)

print("✅ Resumen estadístico completo generado y gráficos guardados en 'graficos_bd_onchain/'")


C:\Users\win10\AppData\Roaming\Python\Python310\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  >>> a[0, 1] = np.nan
C:\Users\win10\AppData\Roaming\Python\Python310\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  >>> a[0, 1] = np.nan
C:\Users\win10\AppData\Roaming\Python\Python310\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  >>> a[0, 1] = np.nan


✅ Resumen estadístico completo generado y gráficos guardados en 'graficos_bd_onchain/'
